In [ ]:
# testing tsnecuda -- not currently working

from tsnecuda import TSNE
# from matplotlib.image import imread

In [ ]:
X = np.random.random((5000, 50))
X_embedded = TSNE(verbose=1).fit_transform(X)
X_embedded.shape

In [ ]:
import os
import random

In [ ]:
import pickle

In [ ]:
from PIL import Image
# from pillow import Image

In [ ]:
from matplotlib.pyplot import imshow

In [ ]:
# check PIL and Pillow version numbers
import PIL
print('Pillow Version:', PIL.__version__)
print('PIL Version:', PIL.VERSION)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model

In [ ]:
# download pre-trained model
# model = keras.applications.densenet.DenseNet201(weights='imagenet', include_top=True)

In [ ]:
model = keras.applications.VGG16(weights='imagenet', include_top=True)

In [ ]:
model.summary()

In [ ]:
def load_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [ ]:
!identify "/home/rte/data/images/1001/converted_256/00000001.jpg"

In [ ]:
print(images[2])

In [ ]:
img, x = load_image("/home/rte/data/images/1001/converted_256/00000001.jpg")
print("shape of x: ", x.shape)
print("data type: ", x.dtype)
plt.imshow(img)

In [ ]:
predictions = model.predict(x)

for _, pred, prob in decode_predictions(predictions)[0]:
    print("predicted %s with probability %0.3f" % (pred, prob))

In [ ]:
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
feat_extractor.summary()

In [ ]:
img, x = load_image("/home/rte/data/images/1001/converted_256/00000001.jpg")
print("shape of x: ", x.shape)
print("data type: ", x.dtype)
plt.imshow(img)

In [ ]:
# set image path here!
# images_path = '/home/rte/data/images/cat/cs.CV/2015/'
images_path = '/home/rte/data/images/random/seq/0-100k/tsne/'

category = images_path.split('/')[6]
year = images_path.split('/')[7]

image_extensions = ['.jpg', '.png', '.jpeg']   # case-insensitive (upper/lower doesn't matter)
max_num_images = 100000

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]

if max_num_images < len(images):
    images = [images[i] for i in sorted(random.sample(range(len(images)), max_num_images))]

print("keeping %d images to analyze" % len(images))

num_x = len(images)

In [ ]:
num_x = len(images)
print(num_x)

In [ ]:
print(category)
print(year)

In [ ]:
print(images[10])

In [ ]:
import time
tic = time.clock()

features = []
for i, image_path in enumerate(images):
    if i % 500 == 0:
        toc = time.clock()
        elap = toc-tic;
        print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(images),elap))
        tic = time.clock()
    img, x = load_image(image_path);
    feat = feat_extractor.predict(x)[0]
    features.append(feat)

print('finished extracting features for %d images' % len(images))

In [ ]:
# have a look at a few plots for comparison
for i in range(10):
    plt.figure(figsize=(16,4))
    plt.plot(features[i])

In [ ]:
from sklearn.decomposition import PCA

features = np.array(features)
pca = PCA(n_components=300)
pca.fit(features)

In [ ]:
pca_features = pca.transform(features)

In [ ]:
import random

# grab a random query image
query_image_idx = int(len(images) * random.random())

# let's display the image
img = image.load_img(images[query_image_idx])
plt.imshow(img)

In [ ]:
from scipy.spatial import distance

similar_idx = [ distance.cosine(pca_features[query_image_idx], feat) for feat in pca_features ]

In [ ]:
idx_closest = sorted(range(len(similar_idx)), key=lambda k: similar_idx[k])[1:6]

In [ ]:
# load all the similarity results as thumbnails of height 100
thumbs = []
for idx in idx_closest:
    img = image.load_img(images[idx])
    img = img.resize((int(img.width * 100 / img.height), 100))
    thumbs.append(img)

# concatenate the images into a single image
concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)

# show the image
plt.figure(figsize = (16,12))
plt.imshow(concat_image)


In [ ]:
def get_closest_images(query_image_idx, num_results=5):
    distances = [ distance.cosine(pca_features[query_image_idx], feat) for feat in pca_features ]
    idx_closest = sorted(range(len(distances)), key=lambda k: distances[k])[1:num_results+1]
    return idx_closest

def get_concatenated_images(indexes, thumb_height):
    thumbs = []
    for idx in indexes:
        img = image.load_img(images[idx])
        img = img.resize((int(img.width * thumb_height / img.height), thumb_height))
        thumbs.append(img)
    concat_image = np.concatenate([np.asarray(t) for t in thumbs], axis=1)
    return concat_image


In [ ]:
# do a query on a random image
query_image_idx = int(len(images) * random.random())
idx_closest = get_closest_images(query_image_idx)
query_image = get_concatenated_images([query_image_idx], 300)
results_image = get_concatenated_images(idx_closest, 200)

# display the query image
plt.figure(figsize = (5,5))
plt.imshow(query_image)
plt.title("query image (%d)" % query_image_idx)

something, x = load_image(images[query_image_idx])
predictions = model.predict(x)

for _, pred, prob in decode_predictions(predictions)[0]:
    print("predicted %s with probability %0.3f" % (pred, prob))

feat = feat_extractor.predict(x)

plt.figure(figsize=(16,4))
plt.plot(feat[0])

# display the resulting images
plt.figure(figsize = (16,12))
plt.imshow(results_image)
plt.title("similar images")

In [ ]:
# check that we still have the features and list of images
for img, f in list(zip(images, pca_features))[0:5]:
    print("image: %s, features: %0.2f,%0.2f,%0.2f,%0.2f... "%(img, f[0], f[1], f[2], f[3]))

In [ ]:
print(len(images))

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
perp = 50
bPCA = True
num_iterations = 2000

In [ ]:
if bPCA:
    X = np.array(pca_features)
else:
    X = np.array(features)
X.shape
tsne = TSNE(n_components=2, learning_rate=150, perplexity=perp, angle=0.2, verbose=2, n_iter=num_iterations).fit_transform(X)

# original command
# tsne = TSNE(n_components=2, learning_rate=150, perplexity=30, angle=0.2, verbose=2).fit_transform(X)

In [ ]:
# normalise points
tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

In [ ]:
from matplotlib.pyplot import imshow

width = 4000
height = 3000
max_dim = 100

full_image = Image.new('RGBA', (width, height))
for img, x, y in zip(images, tx, ty):
    tile = Image.open(img)
    rs = max(1, tile.width/max_dim, tile.height/max_dim)
    tile = tile.resize((int(tile.width/rs), int(tile.height/rs)), Image.ANTIALIAS)
    full_image.paste(tile, (int((width-max_dim)*x), int((height-max_dim)*y)), mask=tile.convert('RGBA'))

plt.figure(figsize = (16,12))
imshow(full_image)

In [ ]:
import datetime
ts = time.time()
st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H-%M-%S')
filename = "tSNE_" + category + "_" + year + "_" + ("feat","pca")[bPCA] + "_n" + str(num_iterations) + "_p" + str(perp) + "_" + st
print(filename)
full_image.save(filename + ".png")

In [ ]:
# !pip install -U git+https://github.com/bmcfee/RasterFairy/ --user
# import rasterfairy

In [ ]:
filename = "tSNE_" + category + "_" + year + "_n" + str(num_iterations) + "_p" + str(perp) + "_" + st
print(filename)

In [ ]:
import pickle


f = "features_" + category + "_" + year + "_vgg_x" + str(num_x) + ".pickle"

print(f)

# WRITE
with open(f, "wb") as write_file:
    pickle.dump(features, write_file)
    write_file.close()

In [ ]:
#READ

f = "features_cs.CV_2012_vgg_x8381.pickle"

load_data = []

with open(f, "rb") as read_file:
    load_data = pickle.load(read_file)
    read_file.close()

In [ ]:
import time

In [ ]:
p = '/home/rte/data/pickles/features_cat_pickle/features_cs.CV_2012_vgg_x8381.pickle' 

category = p.split('_')[3]
year = p.split('_')[4]
print("category: " + category)
print("year: " + str(year))


with open(p, "rb") as read_file:
    images, features = pickle.load(read_file)
    read_file.close()

# check that we still have the features and list of images
print("----- checking images and features -----")
print("length of images: " + str(len(images)))
print("length of features: " + str(len(features)))
for img, f in list(zip(images, features))[0:5]:
    print("image: %s, features: %0.2f,%0.2f,%0.2f,%0.2f... "%(img, f[0], f[1], f[2], f[3]))

if len(images) >= 300:
    features = np.array(features)
    print("----- running pca across features -----")
    pca = PCA(n_components=300)
    pca.fit(features)

    pca_features = pca.transform(features)

    X = np.array(pca_features)
    X.shape
    tsne = TSNE(n_components=2, learning_rate=150, perplexity=perp, angle=0.2, verbose=2, n_iter=num_iterations).fit_transform(X)

    # normalise points
    tx, ty = tsne[:,0], tsne[:,1]
    tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
    ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

    width = 4000
    height = 3000
    max_dim = 100

    full_image = Image.new('RGBA', (width, height))
    for img, x, y in zip(images, tx, ty):
        tile = Image.open(img)
        rs = max(1, tile.width/max_dim, tile.height/max_dim)
        tile = tile.resize((int(tile.width/rs), int(tile.height/rs)), Image.ANTIALIAS)
        full_image.paste(tile, (int((width-max_dim)*x), int((height-max_dim)*y)), mask=tile.convert('RGBA'))

    plt.figure(figsize = (16,12))
    imshow(full_image)

    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H-%M-%S')
    filename = "tSNE_" + category + "_" + year + "_n" + str(num_iterations) + "_p" + str(perp) + "_" + st
    print(filename)
    full_image.save(filename + ".png")


In [ ]:
import time
import datetime
ts = time.time()
st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d_%H-%M-%S')

In [ ]:
# p = '/home/rte/data/pickles/tSNE_pickle_cuda/tSNE_cuda_cs.CV_2012_n2000_p50_2019-06-18_16-35-11.pickle'
p = '/home/rte/data/pickles/tSNE_pickle_cuda/tSNE_cuda_stat.ML_2018_n2000_p50_2019-06-18_16-00-57.pickle' 

# filename = "tSNE_cuda_" + category + "_" + year + "_n" + str(num_iterations) + "_p" + str(perp) + "_" + st
filename = "tSNE_cuda_stat.ML_2018"

with open(p, "rb") as read_file:
    images, tsne = pickle.load(read_file)
    read_file.close()

# normalise points
tx, ty = tsne[:,0], tsne[:,1]
tx = (tx-np.min(tx)) / (np.max(tx) - np.min(tx))
ty = (ty-np.min(ty)) / (np.max(ty) - np.min(ty))

width = 4000
height = 3000
max_dim = 100

print("----- creating image from tiles -----")

full_image = Image.new('RGBA', (width, height))
for img, x, y in zip(images, tx, ty):
    tile = Image.open(img)
    tw = tile.width
    th = tile.height
    print(img)
    print("tile dimensions: x=" + str(tile.width) + " y=" + str(tile.height))
    if tw < 10000 and th < 10000:
        rs = max(1, tw/max_dim, th/max_dim)
        tile = tile.resize((int(tw/rs), int(th/rs)), Image.ANTIALIAS)
        full_image.paste(tile, (int((width-max_dim)*x), int((height-max_dim)*y)), mask=tile.convert('RGBA'))
    else:
        print("tile width or height too big?")

plt.figure(figsize = (16,12))
imshow(full_image)

print("saved file: " + filename + ".png")
full_image.save(filename + ".png")
print("----- finished! file saved -----")